# AIMS

pre processing portfolio dataset:

- rename columns
- retirar simbolos raros
- month number
- create date column.
- one hot encode for channel Enos.

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import pandas as pd

import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [23]:
import sys
sys.path.append("../") 

import utils.paths as path
from utils.paths2 import direcciones

In [24]:
# Crear los de drive
G_raw, G_processed, G_interim, G_external, G_models, G_reports, G_reports_figures = direcciones('temperature')

In [25]:
# read data
df = pd.read_csv(path.data_raw_dir('osb_temperatura.csv'), sep=';', decimal=',', encoding = 'latin1')
# df = pd.read_csv(G_raw/'osb_temperatura.csv', sep=';', decimal=',', encoding = 'latin1')
df.columns= df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(' ', '_')
df.rename(columns={'año':'year',
                   'mes':'month',
                   'temperatura_máxima':'t_max', 
                   'temperatura_mínima':'t_min',
                   'temperatura_promedio':'t_mean',
                   'enos':'fenomeno'
                  }, inplace=True)
df['fenomeno'] = df['fenomeno'].str.replace('ñ','n')
df.head()

,year,month,t_max,t_min,t_mean,fenomeno
0,2007,Enero,26.4,4.4,14.11,Nino
1,2007,Febrero,25.4,-4.6,13.48,Neutro
2,2007,Marzo,24.0,5.1,14.23,Neutro
3,2007,Abril,23.0,6.6,14.75,Neutro
4,2007,Mayo,21.9,5.9,13.26,Neutro


In [26]:
def month_check(df):
    
    '''Esta función toma el nombre de un mes y retorna el número de mes'''
    
    if (df['month'] == 'Enero'):
        return '1'
    elif (df['month'] == 'Febrero'):
        return '2'
    elif (df['month'] == 'Marzo'):
        return '3'
    elif (df['month'] == 'Abril'):
        return '4'
    elif (df['month'] == 'Mayo'):
        return '5'
    elif (df['month'] == 'Junio'):
        return '6'
    elif (df['month'] == 'Julio'):
        return '7'
    elif (df['month'] == 'Agosto'):
        return '8'
    elif (df['month'] == 'Septiembre'):
        return '9'
    elif (df['month'] == 'Octubre'):
        return '10'
    elif (df['month'] == 'Noviembre'):
        return '11'
    elif (df['month'] == 'Diciembre'):
        return '12'
    
df['month_number'] = df.apply(month_check, axis = 1)

In [27]:
cols=['year','month_number']
df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
df['date'] = pd.to_datetime(df['date'])
df.head()

,year,month,t_max,t_min,t_mean,fenomeno,month_number,date
0,2007,Enero,26.4,4.4,14.11,Nino,1,2007-01-01
1,2007,Febrero,25.4,-4.6,13.48,Neutro,2,2007-02-01
2,2007,Marzo,24.0,5.1,14.23,Neutro,3,2007-03-01
3,2007,Abril,23.0,6.6,14.75,Neutro,4,2007-04-01
4,2007,Mayo,21.9,5.9,13.26,Neutro,5,2007-05-01


In [28]:
def fenomeno_check(df):
    
    '''Esta función toma el nombre de un mes y retorna el número de mes'''
    
    if (df['fenomeno'] == 'Neutro'):
        return '0'
    elif (df['fenomeno'] == 'Nino'):
        return '1'
    elif (df['fenomeno'] == 'Nina'):
        return '2'
    
df['fenomeno'] = df.apply(fenomeno_check, axis = 1)

In [31]:
df = df.loc[:,['date','t_max','t_min','t_mean','fenomeno']]
df = df.sort_values(by=['date'], ascending=True)
df.head()

,date,t_max,t_min,t_mean,fenomeno
0,2007-01-01,26.4,4.4,14.11,1
1,2007-02-01,25.4,-4.6,13.48,0
2,2007-03-01,24.0,5.1,14.23,0
3,2007-04-01,23.0,6.6,14.75,0
4,2007-05-01,21.9,5.9,13.26,0


In [32]:
df.to_parquet(path.data_processed_dir('dataset.parquet.gzip'), compression='gzip')
df.to_parquet(G_processed/'dataset.parquet.gzip', compression='gzip')